# [Advent of Code 2017](http://adventofcode.com/2017)

In previous years I've done Advent in Code in Go (2015) and Elixir (2016).
In 2016 I compared solutions with [Cameron](https://github.com/wheat779).

He was completing the questions in Python which required I brush up on Python to be able to assist him with improving his solutions as it had been a number of years since I had touched the language.

With the rise of Python in the Data Science and Machine Learning space I've decided to use Python as my language of choice this year. I'm completing my answers in a Notebook to document the thought processes behind my answers.

To warm up for 2017 I've completed a few of the 2016 questions in Python and begun translating a number of general helper functions to Python from my Elixir answers last year as well as some [Project Euler](https://projecteuler.net) helper functions that *may* come in handy.

I've also stolen and modified a few simple functions from [Peter Norvig's AoC 2016 Notebook](https://github.com/norvig/pytudes/blob/master/ipynb/Advent%20of%20Code.ipynb).

I've placed these helpers in a Notebook named `AoC_Helpers.ipynb` to allow them to be shared between my 2015, 2016 and 2017 Notebooks.

Here, I use the [ipnyb](https://github.com/ipython/ipynb) module to import that Notebook as if it were a normal Python module.

In [1]:
from ipynb.fs.full.aoc_helpers import *

When doing Project Euler I've implemented plenty of graphs and prime sieves so I'll be using more efficient libraries for these in case they come up.

In [2]:
# ! pip install networkx primesieve
import networkx
import primesieve

In [3]:
import re
import numpy as np
import math
import itertools as it


from collections import Counter, defaultdict, namedtuple, deque
from functools   import lru_cache, partial
from itertools   import permutations, combinations, chain, cycle, product, islice
from heapq       import heappop, heappush

As the Helpers module is shared between 2015, 2016 and 2017, I'm partially applying the `Input` function to specify the year.

`Input(day, year) -> partial(Input, year=2017) -> Input(3) == Input(3, 2017)`

This allows me to use `Input(3)` in the 2016 file to get day 3 of 2016, as well as `Input(3)` in the 2017 file to get day 3 of 2017.

In [4]:
Input = partial(Input, year=2017)

## [Day 1](http://adventofcode.com/2017/day/1): Inverse Captcha

Wooo! Time to kick of Advent of Code 2017!

For Part A we need to sum all digits that match the **next digit** in the list. The list is circular, with the end of the list wrapping around to the front.

This means if the front of the list is a 3 and the end of the list is a 3, the end of the list will consider the next digit to be a 3 and thus be added to the sum.

### Part A

In order to compare the value of index $ n $ in the array with $ n + step $ we can create a copy of the array, offset by $ step $. This solves the issue with wrapping around the end of the array and visually is cleaner than indexing.

We then use `zip` to get a tuple of each element of the array along with the offset array to easily compare the two.

In [5]:
one = read_letters(Input(1))

In [6]:
def captcha(nums, part_b=False):
    step = len(nums) // 2 if part_b else 1
    return sum(int(a) for a, b in zip(nums, nums[step:] + nums[:step]) if a == b)

In [7]:
assert captcha('1221') == 3
assert captcha('1111') == 4
assert captcha('1234') == 0
assert captcha('91212129') == 9

In [8]:
captcha(one)

1141

### Part B
For Part B, instead of considering the next number we want to consider the digit **halfway around** the circular list.

I've added a `part_b` argument which changes the `step` size from 1 to half the length of the input.

In [9]:
assert captcha('1212', part_b=True) == 6
assert captcha('1221', part_b=True) == 0
assert captcha('123425', part_b=True) == 4
assert captcha('123123', part_b=True) == 12
assert captcha('12131415', part_b=True) == 4

In [10]:
captcha(one, part_b=True)

950

In case more array rotations end up coming up in the coming days I'm going to define a function to allow me to easily rotate an array a given number of steps.

In [11]:
def rotate(iterable, steps=1):
    "Rotate the iterable a given number of steps."
    steps = steps % len(iterable)
    return iterable[steps:] + iterable[:steps] 

In [12]:
assert rotate([1, 2, 3, 4, 5]) == [2, 3, 4, 5, 1]
assert rotate([1, 2, 3, 4, 5], steps=-1) == [5, 1, 2, 3, 4]
assert rotate([1, 2, 3, 4, 5], steps=3)  == [4, 5, 1, 2, 3]
assert rotate([1, 2, 3, 4, 5], steps=10) == [1, 2, 3, 4, 5]

## [Day 2](http://adventofcode.com/2017/day/2): Corruption Checksum

At first glance this seems even more straightforward than day 1.

Given a tab separated spreadsheet we need to compute a checksum!

For each row, find the $ min $ and $ max $ values and compute the difference between them.

The checksum is the sum of the result of each row.

In [13]:
def int_lines(lines):
    "Parse ints of each subarray of the input array"
    return list(map(parse_ints, lines))

In [14]:
spreadsheet = int_lines(Input(2))

The difference between the $ min $ and $ max $ amplitude of a waveform is known as the Peak to Peak value.

In [15]:
def ptp(iterable):
    "The difference between the maximum and minimum values"
    return max(iterable) - min(iterable)

In [16]:
assert ptp([5, 1, 9, 5]) == 8

In [17]:
sum(ptp(row) for row in spreadsheet)

41887

### Part B

Instead of summing the min and max values from each row we now must find the two numbers for which $a \bmod b = 0$, keeping the result of $a\div b$

In [18]:
def evenly_divisible(ints):
    "Return the evenly divisible combinations of numbers in the input"
    for (a, b) in combinations(sorted(ints), 2):
        if b % a == 0: yield (b, a)

In [19]:
assert next(evenly_divisible([5, 9, 2, 8])) == (8, 2)

In [20]:
sum(a // b for row in spreadsheet for (a, b) in evenly_divisible(row))

226

## [Day 3](http://adventofcode.com/2017/day/3): Spiral Memory

You come across an experimental new kind of memory stored on an infinite two-dimensional grid.

Each square on the grid is allocated in a spiral pattern starting at a location marked 1 and then counting up while spiraling outward. For example, the first few squares are allocated like this:

```
7   16  15  14  13
18   5   4   3  12
19   6   1   2  11
20   7   8   9  10
21  22  23---> ...
```

Observing the number of spaces moved before each turn we can see a clear pattern:

```
4   4   4   4  3
4   2   2   1  3
4   2   0   1  3
4   2   3   3  3
4   5   5--> ...
```

First, we move one space to the right. We then turn to the left and move one space upwards.  
Now we turn and move two spaces to the left. We turn, and move two spaces.  
We turn, and move three spaces. We turn, and move three spaces.  
We turn, and move four spa...  

For each iteration of the spiral we move forward $ n $ spaces for two turns, we then move $ n+1 $ spaces for two turns, incrementing $ n $ by $ 1 $ every two turns.


In [21]:
goal = 347991

In [22]:
def spiral(part_b=False):
    matrix, position, facing = defaultdict(int), Point(0, 0), Point(1, 0)
    max_steps, stepped, turn_count, num = 1, 0, 0, 0

    while True:
        if part_b:
            num = sum(matrix[n] for n in neighbors8(position))
            if not num: num = 1
        else:
            num += 1

        matrix[position] = num
        yield (position, num)

        position += facing
        stepped += 1
        
        if stepped == max_steps:
            stepped = 0
            turn_count += 1
            # We increase the distance traveled every two turns
            if turn_count == 2:
                max_steps += 1
                turn_count = 0
            # Rotate to the left by swapping x and y and negating one of them.
            facing = Point(facing.y, -facing.x)

### Part A
For Part A we're asked to find the cityblock distance from 0,0 to the point that holds our input value of 347991

In [23]:
def spiral_distance(n): return next(cityblock_distance(pos) for (pos, val) in spiral() if val == n)

In [24]:
assert spiral_distance(1) == 0
assert spiral_distance(12) == 3
assert spiral_distance(23) == 2
assert spiral_distance(1024) == 31

In [25]:
spiral_distance(goal)

480

### Part B

For Part B we modify how values are calculated. Instead of the spiral being a monotonically increasing value we instead sum the values of all of the squares neighbors at the time of calculation.  
I've added a `part_b` argument to the `spiral` generator to modify its number generation to follow the required format.

```
147  142  133  122   59
304    5    4    2   57
330   10    1    1   54
351   11   23   25   26
362  747  806--->   ...
```

We're asked to find the value in the spiral that immediately follows our input value.

In [26]:
def next_spiral_value(n): return next(val for (pos, val) in spiral(part_b=True) if val > n)

In [27]:
assert next_spiral_value(23) == 25
assert next_spiral_value(308) == 330
assert next_spiral_value(482) == 747

In [28]:
next_spiral_value(goal)

349975

## [Day 4](http://adventofcode.com/2017/day/4): High-Entropy Passphrases


### Part A

To ensure security, a valid passphrase must contain no duplicate words.

For example:

aa bb cc dd ee is valid.  
aa bb cc dd aa is not valid - the word aa appears more than once.  
aa bb cc dd aaa is valid - aa and aaa count as different words.  

In [29]:
passphrases = read_words(Input(4))

In [30]:
def all_unique(iterable):
    return len(iterable) == len(set(iterable))

The easiest way to determine that every word in a list is unique is to create a set from the list and compare the lengths.

In [31]:
sum(1 for phrase in passphrases if all_unique(phrase))

455

### Part B

For part B a passphrase must contain no duplicate anagrams.

abcde fghij is a valid passphrase.  
abcde xyz ecdab is not valid - the letters from the third word can be rearranged to form the first word.  
a ab abc abd abf abj is a valid passphrase, because all letters need to be used when forming another word.  
iiii oiii ooii oooi oooo is valid.  
oiii ioii iioi iiio is not valid - any of these words can be rearranged to form any other word.  


In [32]:
def sort_word(word):
    return cat(sorted(word))

In [33]:
sorted_passphrases = [[sort_word(word) for word in phrase] for phrase in passphrases]

In [34]:
sum(1 for phrase in sorted_passphrases if all_unique(phrase)) 

186